In [108]:
!pip install --upgrade pip
!pip install pandas
!pip install numpy

Requirement already up-to-date: pip in c:\program files\python35\lib\site-packages (20.2.3)


In [109]:
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings('ignore')

In [110]:
problem = pd.read_csv('problem.csv')
user = pd.read_csv('user.csv')
target_user = pd.read_csv('koosaga.csv')



In [111]:
target_list = target_user[target_user['정답여부']==False]
target_list.columns
target_problem = np.array(target_list['문제번호'].tolist())
print(target_problem)

[11937 11938 11939 11940]


In [112]:
level_array = ['Bronze','Silver','Gold','Platinum','Diamond','Ruby']

In [113]:
#장르 유사도에 따라 영화를 추천하는 함수
#함수명 : find_sim_movie()
#movies_df DataFrame : 기반 데이터
#genre_sim_sorted_ind : 레코드별 장르 코사인 유사도 인덱스
#DataFrame : 고객이 선정한 추천 기준이 되는 영화제목, 추천할 영화 건수를 입력하면 추천 영화 정보를 가짐
def find_sim_problem(df, sorted_ind, title_name, top_n = 10):
    
    title_problem = df[df['문제번호']== int(title_name)]
    
    title_index = title_problem.index.values
    similar_indexes = sorted_ind[title_index,:(top_n)]
    
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [114]:
def level_set(df,target_problem,level_array):
    level_idx = df.index[df['문제번호']==int(target_problem)]
    level = level_array.index(df.loc[level_idx[0]]['난이도'].split(' ')[0])
    print(level)
    df['level'],df['temp']= df['난이도'].str.split(' ',1).str
    for i, row in df.iterrows() :
        if df.at[i,'level'] == 'Bronze':  # (1)
            df.at[i,'평균시도'] = (abs(0-level))*df.at[i,'평균시도']            
        elif df.at[i,'level'] == 'Silver':  # (1)
            df.at[i,'평균시도'] = (abs(1-level))*df.at[i,'평균시도']
        elif df.at[i,'level'] == 'Gold':  # (1)
            df.at[i,'평균시도'] = (abs(2-level))*df.at[i,'평균시도']
        elif df.at[i,'level'] == 'Platium':  # (1)
            df.at[i,'평균시도'] = (abs(3-level))*df.at[i,'평균시도']
        elif df.at[i,'level'] == 'Diamond':  # (1)
            df.at[i,'평균시도'] = (abs(4-level))*df.at[i,'평균시도']
        elif df.at[i,'level'] == 'Ruby':  # (1)
            df.at[i,'평균시도'] = (abs(5-level))*df.at[i,'평균시도']
        else:
            df.at[i,'평균시도'] = 0

In [115]:
def weighted_vote_average(record):
    v = record['해결']
    R = record['평균시도']

    return  (v/(v+m)) * R + (v/(v+m)) * C

In [116]:
def find_sim_problem(df, sorted_ind, title_name, top_n =3):
    title_problem = df[df['문제번호'] == int(title_name)]
    title_index = title_problem.index.values
    #top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexs = similar_indexes.reshape(-1)
    #기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes!=title_index]
    #top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df

In [117]:
def problem_recom(target_problem,problem,level_array):
    
    problem_df = problem[['문제번호','난이도','분류','해결','평균시도']]
    pd.set_option('max_colwidth',100)
    #각각의 장르와 그 장르의 id를 추출해 낼 수 있다
    problem_df[['분류','난이도']][:1]
    from sklearn.feature_extraction.text import CountVectorizer

    #CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
    count_vect  = CountVectorizer(min_df = 0, ngram_range=(1,2))
    genre_mat = count_vect.fit_transform(problem_df['분류'])

    from sklearn.metrics.pairwise import cosine_similarity
    genre_sim = cosine_similarity(genre_mat, genre_mat) 

    genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]

    similar_movies = find_sim_problem(problem_df, genre_sim_sorted_ind,target_problem, 40)
    similar_movies[['문제번호','평균시도','해결','난이도','분류']]
    problem_df[['문제번호','난이도','평균시도']].sort_values('평균시도',ascending=True)[:10]
    level_set(problem_df,target_problem,level_array)

    C = problem_df['평균시도'].mean()
    m = problem_df['해결'].quantile(0.6)
    
    #원래 점수에 
    percentile = 0.6
    m = problem_df['해결'].quantile(0.6)
    C = problem_df['평균시도'].mean()

    problem['weighted_vote'] = problem_df.apply(weighted_vote_average,axis=1)
 
    problem = problem.drop_duplicates(['문제번호'],keep='last')
    #새로 부여된 weighted_vote 평점이 높은 순으로 상위 10개 추출
    problem[['문제번호','제목','난이도','평균시도','weighted_vote','해결']].sort_values(
        'weighted_vote',ascending=False)[:10] 

    similar_problem = find_sim_problem(problem,genre_sim_sorted_ind,target_problem,10)
    similar_problem[['문제번호','난이도','제목','weighted_vote']]
    return similar_problem


In [118]:
Problem = pd.DataFrame(columns=['문제번호','난이도','제목','weighted_vote'])
for i in range(0,len(target_problem)):
    print(i)
    Problem.append(problem_recom(target,problem,level_array))

Problem.drop_duplicates()


0


ValueError: 'Platinum' is not in list